In [13]:
from kerax.layers import Conv2D, Dense, Flatten, Input, Activation, Add
from kerax.models import Model
from jax.random import PRNGKey
from jax import random
import gc
gc.enable()

In [14]:
img = random.normal(key=PRNGKey(3), shape=(1, 28, 28, 1))

train_x = random.normal(key=PRNGKey(3), shape=(128, 28, 28, 1))
train_y = random.normal(key=PRNGKey(34), shape=(128, 1))

In [15]:
inputs = Input((28, 28, 1))
conv1 = Conv2D(64, 3, key=PRNGKey(100), padding='same')(inputs)
act1 = Activation('relu')(conv1)
conv3 = Conv2D(128, 3, padding='same', key=PRNGKey(104))(act1)
act2 = Activation('relu')(conv3)
conv4 = Conv2D(128, 3, padding='same', key=PRNGKey(105))(act2)
add = Add()([conv4, conv3])
flatten = Flatten()(add)
dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(1)(dense1)


In [16]:
model = Model(input=inputs, output=dense2)

In [17]:
model.compile(loss='mse', optimizer='adam')

In [18]:
model.fit(train_x, train_y, epochs=2, batch_size=32)

100%|████████████████████████████████████████| 32/32 [00:23<00:00,  1.39it/s, loss=0.00201192, remaining_epochs=0]  
